In [ ]:

    

def prompt_ner(self, txt, substrings):

        # Set up a parser + inject instructions into the prompt template.
        parser = PydanticOutputParser(pydantic_object=IcdList)

        format_instructions = parser.get_format_instructions()
        # coding_instructions = "What is the correct ICD-10 code for the substring. If you think the correct code is not listed, provide your best code suggestion in the json field 'code', always follow the format instructions."
        behaviour_instructions = "You are an expert in medical ICD coding, teaching peers the highest level of coding possible."
        system_message_prompt = SystemMessagePromptTemplate.from_template("{behaviour_instructions} {format_instructions}\n")
        human_message_prompt = HumanMessagePromptTemplate.from_template("For each of items in the lists, add additional information in Spanish to preform an medical code lookup for '{substring}', use {format_instructions} in markdown! Extract from the following text {txt}. The number of list items of the output should match the input, ")

        chat_prompt = ChatPromptTemplate(
            messages=[system_message_prompt,human_message_prompt], 
            input_variables=["substring", "txt"],
            partial_variables={"behaviour_instructions": behaviour_instructions, "format_instructions": format_instructions,}
        )
        _input = chat_prompt.format_prompt( substring=substrings, txt=txt)
        logging.info(_input.to_messages())
        output = self.chat(_input.to_messages())
        logging.info(output.content)
        print(output.content)
        try:
            json_substrings = parser.parse(output.content)
            print(json_substrings)
        except Exception as e: 
            logging.error(e)            
        return json_substrings